## Import Modules

In [18]:

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
from sklearn import metrics
from sklearn.metrics import roc_auc_score
import time

In [19]:
from sklearn import svm, tree
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.utils import to_categorical
import joblib
import numpy as np
from numpy import mean,std
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

## Load Data for Layer 1

In [20]:
from load_data_layer1 import load_training_data, load_testing_data

# X, y = load_training_data()
# X_test, y_test = load_testing_data()

X, y = np.load('x_train_layer1.npy'), np.load('y_train_layer1.npy')
X_test, y_test = np.load('x_test_layer1.npy'), np.load('y_test_layer1.npy')

In [21]:
class evaluation_result:
    def __init__(self, time, precision, recall, f1, accuracy, AUC, specificity, sensitivity):
        self.time = time
        self.precision = precision
        self.recall = recall
        self.f1 = f1
        self.accuracy = accuracy
        self.AUC = AUC
        self.precision = precision
        self.specificity = specificity
        self.sensitivity = sensitivity

In [22]:
def spec_sens(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    return [specificity, sensitivity]

In [23]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, shuffle=True)
kf.get_n_splits(X)

10

In [24]:
def CrossValidation(model, epochs):
    i = 0
    training_time = []
    precision_list = []
    recall_list = []
    acc_list = []
    f1_list = []
    spec_list = []
    sens_list = []
    auc_list = []
    
    for train_index, val_index in kf.split(X):
        # print("TRAIN:", train_index, "TEST:", val_index)
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

    
        if epochs == None:
            t = time.time()
            model.fit(X_train, y_train)
            training_time.append((time.time() - t))
            y_pred = model.predict(X_val)
            y_true = y_val

        else:
            le.fit(y_train)
            y_train = le.transform(y_train)
            y_train = to_categorical(y_train)
            y_val = le.transform(y_val)
            y_val = to_categorical(y_val)

            t = time.time()
            model.fit(X_train, y_train, epochs=epochs, verbose=0)
            training_time.append((time.time() - t))
            
            value = model.predict(X_val)
            y_pred = np.argmax(value,axis=1)
            y_true = np.argmax(y_val,axis=1)
            

        precision_list.append(precision_score(y_true, y_pred))
        recall_list.append(recall_score(y_true, y_pred))
        f1_list.append(f1_score(y_true, y_pred))
        acc_list.append(accuracy_score(y_true, y_pred))
        auc_list.append(roc_auc_score(y_true, y_pred))
        sens_list.append(spec_sens(y_true, y_pred)[0])
        spec_list.append(spec_sens(y_true, y_pred)[1])

        i+=1
    result = evaluation_result(training_time, precision_list, recall_list, f1_list, acc_list, auc_list, spec_list, sens_list)
    return model, result

## Define 3 classification model 

In [25]:
def clf_NN_model():
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(256,)))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [26]:
DT = tree.DecisionTreeClassifier(ccp_alpha=0.001, criterion='entropy', max_depth=15, random_state=42)
SVM = svm.SVC(C=100, gamma=1,kernel='rbf')
NN = clf_NN_model()

In [27]:
DT, DT_result = CrossValidation(DT,None)
print('Decision Tree for Healthy Mean Accuracy: %.3f ± %.3f)' % (mean(DT_result.accuracy), std(DT_result.accuracy)))
print('Decision Tree for Healthy Mean Sensitivity: %.3f ± %.3f)' % (mean(DT_result.sensitivity), std(DT_result.sensitivity)))
print('Decision Tree for Healthy Mean Specificity: %.3f ± %.3f)' % (mean(DT_result.specificity), std(DT_result.specificity)))
print('Decision Tree for Healthy Mean Precision: %.3f ± %.3f)' % (mean(DT_result.precision), std(DT_result.precision)))
print('Decision Tree for Healthy Mean Recall: %.3f ± %.3f)' % (mean(DT_result.recall), std(DT_result.recall)))
print('Decision Tree for Healthy Mean F Measure: %.3f ± %.3f)' % (mean(DT_result.f1), std(DT_result.f1)))
print('Decision Tree for Healthy Mean AUC: %.3f ± %.3f)' % (mean(DT_result.AUC), std(DT_result.AUC)))
print('Decision Tree for Healthy Mean Training Time: %.3f ± %.3f)' % (mean(DT_result.time), std(DT_result.time)))

Decision Tree for Healthy Mean Accuracy: 0.898 ± 0.010)
Decision Tree for Healthy Mean Sensitivity: 0.887 ± 0.021)
Decision Tree for Healthy Mean Specificity: 0.910 ± 0.019)
Decision Tree for Healthy Mean Precision: 0.890 ± 0.018)
Decision Tree for Healthy Mean Recall: 0.910 ± 0.019)
Decision Tree for Healthy Mean F Measure: 0.899 ± 0.009)
Decision Tree for Healthy Mean AUC: 0.898 ± 0.010)
Decision Tree for Healthy Mean Training Time: 1.649 ± 0.068)


In [28]:
SVM, SVM_result = CrossValidation(SVM,None)
print('SVM for Healthy Mean Accuracy: %.3f ± %.3f)' % (mean(SVM_result.accuracy), std(SVM_result.accuracy)))
print('SVM for Healthy Mean Sensitivity: %.3f ± %.3f)' % (mean(SVM_result.sensitivity), std(SVM_result.sensitivity)))
print('SVM for Healthy Mean Specificity: %.3f ± %.3f)' % (mean(SVM_result.specificity), std(SVM_result.specificity)))
print('SVM for Healthy Mean Precision: %.3f ± %.3f)' % (mean(SVM_result.precision), std(SVM_result.precision)))
print('SVM for Healthy Mean Recall: %.3f ± %.3f)' % (mean(SVM_result.recall), std(SVM_result.recall)))
print('SVM for Healthy Mean F Measure: %.3f ± %.3f)' % (mean(SVM_result.f1), std(SVM_result.f1)))
print('SVM for Healthy Mean AUC: %.3f ± %.3f)' % (mean(SVM_result.AUC), std(SVM_result.AUC)))
print('SVM for Healthy Mean Training Time: %.3f ± %.3f)' % (mean(SVM_result.time), std(SVM_result.time)))

SVM for Healthy Mean Accuracy: 0.963 ± 0.011)
SVM for Healthy Mean Sensitivity: 0.952 ± 0.015)
SVM for Healthy Mean Specificity: 0.974 ± 0.011)
SVM for Healthy Mean Precision: 0.953 ± 0.015)
SVM for Healthy Mean Recall: 0.974 ± 0.011)
SVM for Healthy Mean F Measure: 0.963 ± 0.010)
SVM for Healthy Mean AUC: 0.963 ± 0.011)
SVM for Healthy Mean Training Time: 1.314 ± 0.058)


In [29]:
NN, NN_result = CrossValidation(NN,100)
print('NN for Healthy Mean Accuracy: %.3f ± %.3f)' % (mean(NN_result.accuracy), std(NN_result.accuracy)))
print('NN for Healthy Mean Sensitivity: %.3f ± %.3f)' % (mean(NN_result.sensitivity), std(NN_result.sensitivity)))
print('NN for Healthy Mean Specificity: %.3f ± %.3f)' % (mean(NN_result.specificity), std(NN_result.specificity)))
print('NN for Healthy Mean Precision: %.3f ± %.3f)' % (mean(NN_result.precision), std(NN_result.precision)))
print('NN for Healthy Mean Recall: %.3f ± %.3f)' % (mean(NN_result.recall), std(NN_result.recall)))
print('NN for Healthy Mean F Measure: %.3f ± %.3f)' % (mean(NN_result.f1), std(NN_result.f1)))
print('NN for Healthy Mean AUC: %.3f ± %.3f)' % (mean(NN_result.AUC), std(NN_result.AUC)))
print('NN for Healthy Mean Training Time: %.3f ± %.3f)' % (mean(NN_result.time), std(NN_result.time)))

NN for Healthy Mean Accuracy: 0.986 ± 0.010)
NN for Healthy Mean Sensitivity: 0.986 ± 0.013)
NN for Healthy Mean Specificity: 0.986 ± 0.013)
NN for Healthy Mean Precision: 0.986 ± 0.013)
NN for Healthy Mean Recall: 0.986 ± 0.013)
NN for Healthy Mean F Measure: 0.986 ± 0.010)
NN for Healthy Mean AUC: 0.986 ± 0.010)
NN for Healthy Mean Training Time: 19.639 ± 1.764)


In [30]:
y_pred_DT = DT.predict(X_test)
y_true_DT = y_test

print('Decision Tree for Healthy Accuracy: %.3f ' % (accuracy_score(y_true_DT, y_pred_DT)))
print('Decision Tree for Healthy Sensitivity: %.3f ' % (spec_sens(y_true_DT, y_pred_DT)[0]))
print('Decision Tree for Healthy Specificity: %.3f ' % (spec_sens(y_true_DT, y_pred_DT)[1]))
print('Decision Tree for Healthy Precision: %.3f ' % (precision_score(y_true_DT, y_pred_DT)))
print('Decision Tree for Healthy F Measure: %.3f ' % (f1_score(y_true_DT, y_pred_DT)))
print('Decision Tree for Healthy AUC: %.3f ' % (roc_auc_score(y_true_DT, y_pred_DT)))

Decision Tree for Healthy Accuracy: 0.862 
Decision Tree for Healthy Sensitivity: 0.871 
Decision Tree for Healthy Specificity: 0.852 
Decision Tree for Healthy Precision: 0.869 
Decision Tree for Healthy F Measure: 0.861 
Decision Tree for Healthy AUC: 0.862 


In [31]:
y_pred_SVM = SVM.predict(X_test)
y_true_SVM = y_test

print('SVM for Healthy Accuracy: %.3f ' % (accuracy_score(y_true_SVM, y_pred_SVM)))
print('SVM for Healthy Sensitivity: %.3f ' % (spec_sens(y_true_SVM, y_pred_SVM)[0]))
print('SVM for Healthy Specificity: %.3f ' % (spec_sens(y_true_SVM, y_pred_SVM)[1]))
print('SVM for Healthy Precision: %.3f ' % (precision_score(y_true_SVM, y_pred_SVM)))
print('SVM for Healthy F Measure: %.3f ' % (f1_score(y_true_SVM, y_pred_SVM)))
print('SVM for Healthy AUC: %.3f ' % (roc_auc_score(y_true_SVM, y_pred_SVM)))

SVM for Healthy Accuracy: 0.934 
SVM for Healthy Sensitivity: 0.957 
SVM for Healthy Specificity: 0.911 
SVM for Healthy Precision: 0.955 
SVM for Healthy F Measure: 0.933 
SVM for Healthy AUC: 0.934 


In [32]:
y_pred_NN = NN.predict(X_test)
y_pred_NN = np.argmax(y_pred_NN,axis=1)

le.fit(y_test)
y_test_NN = le.transform(y_test)
y_test_NN = to_categorical(y_test_NN)
y_true_NN = np.argmax(y_test_NN,axis=1)

print('NN for Healthy Accuracy: %.3f ' % (accuracy_score(y_true_NN, y_pred_NN)))
print('NN for Healthy Sensitivity: %.3f ' % (spec_sens(y_true_NN, y_pred_NN)[0]))
print('NN for Healthy Specificity: %.3f ' % (spec_sens(y_true_NN, y_pred_NN)[1]))
print('NN for Healthy Precision: %.3f ' % (precision_score(y_true_NN, y_pred_NN)))
print('NN for Healthy F Measure: %.3f ' % (f1_score(y_true_NN, y_pred_NN)))
print('NN for Healthy AUC: %.3f ' % (roc_auc_score(y_true_NN, y_pred_NN)))

NN for Healthy Accuracy: 0.956 
NN for Healthy Sensitivity: 0.979 
NN for Healthy Specificity: 0.933 
NN for Healthy Precision: 0.978 
NN for Healthy F Measure: 0.955 
NN for Healthy AUC: 0.956 


# Save the best classification model in Layer 1

In [33]:
NN.save('model/NN')

INFO:tensorflow:Assets written to: model/NN\assets
